In [1]:
import pathlib
import pandas as pd 
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

import nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playerdashboardbyclutch, playergamelogs, commonplayerinfo, teamplayeronoffdetails, teamgamelogs


In [2]:
class Team(object):
    def __init__(self, TeamID):
        
        self.TeamID =  TeamID 
        self.TeamName = teams.find_team_name_by_id(team_id=self.TeamID)['full_name']

        self.StartYear = 2009 #First year data is avalible
        self.LastYear = 2021 #Current year
        
        
        #team abreviations and corresponding codes for data proccessing
        self.team_abbreviation_codes = {
            "ATL": 1,
            "BKN": 2,
            "BOS": 3,
            "CHA": 4,
            "CHI": 5,
            "CLE": 6,
            "DAL": 7,	
            "DEN": 8,
            "DET": 9,
            "GSW": 10,	
            "HOU": 11,	
            "IND": 12,
            "LAC": 13,	
            "LAL": 14,	
            "MEM": 15, 	
            "MIA": 16,	
            "MIL": 17,	
            "MIN": 18,	
            "NOP": 19,
            "NOH": 19,	
            "NYK": 20,	
            "OKC": 21,	
            "ORL": 22,	
            "PHI": 23,	
            "PHX": 24,	
            "POR": 25,	
            "SAC": 26,	
            "SAS": 27,	
            "TOR": 28,	
            "UTA": 29,	
            "WAS": 30
        }

        
    #Complile the Team Data
    def compile_active_team_data(self):
        

        matchup_raw_data = []
    
        team_season_point_average_raw_data = []
        team_points_raw_data = []
        opponent_season_point_average_raw_data = []
        opponent_season_points_raw_data = []
        win_loss_raw_data = []
        
        print(self.StartYear, self.LastYear)
        for year in range((self.StartYear), (self.LastYear)):
            print("-------------------------------------------------")
            season = str(year) + "-" + str(((year+1)-2000))
            teamLogs = teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable=season)
            print(season)
            
            win_loss_api_data = teamLogs.get_data_frames()[0]["WL"]
            matchup_api_data = teamLogs.get_data_frames()[0]["MATCHUP"]
            team_season_points_api_data=teamLogs.get_data_frames()[0]["PTS"]
        

            print(teamLogs.get_data_frames()[0])


            team_season_points = []
            
            for game in range(0, len(teamLogs.get_data_frames()[0])):
        
                    win_loss_raw_data.append(win_loss_api_data[game])
                    
                    team_points_raw_data.append(team_season_points_api_data[game])
                    
                    team_season_points.append(team_season_points_api_data[game])
                    if game == 0:
                        team_season_point_average_raw_data.append(teamLogs.get_data_frames()[0]["PTS"][game])
                    else:
                        team_season_point_average_raw_data.append((sum(team_season_points)/len(team_season_points)))
                        
                    
                    self.team_abbreviation = teams.find_team_name_by_id(team_id=self.TeamID)['abbreviation']
                    matchup_api_data[game] = matchup_api_data[game].replace('NOH', 'NOP')
                    matchup_api_data[game] = matchup_api_data[game].replace('NJN', 'BKN')
                
                    

                    try:   
                        
                        matchup_api_data[game] = matchup_api_data[game].replace(self.team_abbreviation, "")
                        
                    except:
                        print("Unexpected Error with removing player team")
                    try:
                        matchup_api_data[game] = matchup_api_data[game].replace(" @ ", "")
                        matchup_api_data[game] = matchup_api_data[game].replace(" vs. ", "")
                    except:
                        print("Error")
                    
                    
                    
                    opponentAbbreviation = matchup_api_data[game]
                    
                    matchup_api_data[game] = self.team_abbreviation_codes[opponentAbbreviation]
                    matchup_raw_data.append(matchup_api_data[game])
                    
                    opponentID = int(teams.find_team_by_abbreviation(opponentAbbreviation)["id"])
                    if (game%10==0):
                        print(game)
                        
                    opponentLogs = teamgamelogs.TeamGameLogs(team_id_nullable=opponentID, season_nullable=season)
                    opponent_season_points_api_data = opponentLogs.get_data_frames()[0]['PTS']
                    if game == 0:
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data[game])
                    elif game<(len(opponentLogs.get_data_frames()[0]['PTS'])):
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data.head(game).mean())
                    else:
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data.mean())
                                            
                    time.sleep(1)
                    

        #Lengths of lists used for troublshooting
        print("the lengths")
        print(len(win_loss_raw_data))
        print(len(matchup_raw_data))
        print(len(team_season_point_average_raw_data))
        print(len(opponent_season_point_average_raw_data))
        
        #adding the lists to the team data dictionary to be made into a dataframe
        team_data = {
        "Win/Loss": win_loss_raw_data,
        #"Points Scored": team_points_raw_data,
        "Matchup": matchup_raw_data,
        "Team Point Average" : team_season_point_average_raw_data,
        "Opponent Team Point Average": opponent_season_point_average_raw_data 
        }

        team_dataFrame = pd.DataFrame(data=team_data)
        print(team_dataFrame) #print for troublshooting

        return(team_dataFrame)


    def process_active_team_data(self,data_for_processing):
        data = data_for_processing
        opponent_processing = data.pop('Matchup')

        data["ATL"] = (opponent_processing == 1)*1.0
        data["BKN"] = (opponent_processing == 2)*1.0
        data["BOS"] = (opponent_processing == 3)*1.0
        data["CHA"] = (opponent_processing == 4)*1.0
        data["CHI"] = (opponent_processing == 5)*1.0
        data["CLE"] = (opponent_processing == 6)*1.0
        data["DAL"] = (opponent_processing == 7)*1.0
        data["DEN"] = (opponent_processing == 8)*1.0
        data["DET"] = (opponent_processing == 9)*1.0
        data["GSW"] = (opponent_processing == 10)*1.0
        data["HOU"] = (opponent_processing == 11)*1.0
        data["IND"] = (opponent_processing == 12)*1.0
        data["LAC"] = (opponent_processing == 13)*1.0
        data["LAL"] = (opponent_processing == 14)*1.0
        data["MEM"] = (opponent_processing == 15)*1.0
        data["MIA"] = (opponent_processing == 16)*1.0
        data["MIL"] = (opponent_processing == 17)*1.0
        data["MIN"] = (opponent_processing == 18)*1.0
        data["NOP"] = (opponent_processing == 19)*1.0
        data["NYK"] = (opponent_processing == 20)*1.0
        data["OKC"] = (opponent_processing == 21)*1.0
        data["ORL"] = (opponent_processing == 22)*1.0
        data["PHI"] = (opponent_processing == 23)*1.0
        data["PHX"] = (opponent_processing == 24)*1.0
        data["POR"] = (opponent_processing== 25)*1.0
        data["SAC"] = (opponent_processing == 26)*1.0
        data["SAS"] = (opponent_processing == 27)*1.0
        data["TOR"] = (opponent_processing == 28)*1.0
        data["UTA"] = (opponent_processing == 29)*1.0
        data["WAS"] = (opponent_processing == 30)*1.0		


        data = data.drop(columns=[self.team_abbreviation])

        return(data)
       
    #Get the Team data for outside calls
    def get_active_team_data(self):
        compiled_data = self.compile_active_team_data()
        return(self.process_active_team_data(compiled_data))
    def compile_data_for_prediction(self, opponentAbbreviation_for_prediction):
        print(22222)
        season = "2020-21"
        teamLogs = teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable=season)
        team_season_points_api_data = teamLogs.get_data_frames()[0]["PTS"]
        print(team_season_points_api_data)
        team_season_point_average = [team_season_points_api_data.mean()]
        
        print(77777777777777777777777777777777777777777, team_season_point_average)
        opponentID = int(teams.find_team_by_abbreviation(opponentAbbreviation_for_prediction)["id"])
        opponentLogs = teamgamelogs.TeamGameLogs(team_id_nullable=opponentID, season_nullable=season)
        opponent_season_points_api_data = opponentLogs.get_data_frames()[0]["PTS"]
        opponent_season_point_average = [opponent_season_points_api_data.mean()]

        opponent_code = self.team_abbreviation_codes[opponentAbbreviation_for_prediction]
        
        prediction_data = {
        "Matchup": [opponent_code],
        "Team Point Average" : team_season_point_average,
        "Opponent Team Point Average": opponent_season_point_average
        }
        prediction_dataframe = pd.DataFrame(data=prediction_data)
        
        print(prediction_dataframe)
        return(prediction_dataframe)
        
    def get_data_for_prediction(self, opponentAbbreviation_for_prediction):
        compiled_data = self.compile_data_for_prediction(opponentAbbreviation_for_prediction)
        print("Here is the result", )
        return(self.process_active_team_data(compiled_data))
    



In [3]:
team =Team(1610612752) 
test = team.get_active_team_data() 




    


print(test)

2009 2021
-------------------------------------------------
2009-10
   SEASON_YEAR     TEAM_ID TEAM_ABBREVIATION        TEAM_NAME     GAME_ID  \
0      2009-10  1610612752               NYK  New York Knicks  0020901226   
1      2009-10  1610612752               NYK  New York Knicks  0020901206   
2      2009-10  1610612752               NYK  New York Knicks  0020901199   
3      2009-10  1610612752               NYK  New York Knicks  0020901174   
4      2009-10  1610612752               NYK  New York Knicks  0020901158   
..         ...         ...               ...              ...         ...   
77     2009-10  1610612752               NYK  New York Knicks  0020900064   
78     2009-10  1610612752               NYK  New York Knicks  0020900047   
79     2009-10  1610612752               NYK  New York Knicks  0020900034   
80     2009-10  1610612752               NYK  New York Knicks  0020900019   
81     2009-10  1610612752               NYK  New York Knicks  0020900009   

       

10
20
30
40
50
60
-------------------------------------------------
2012-13
   SEASON_YEAR     TEAM_ID TEAM_ABBREVIATION        TEAM_NAME     GAME_ID  \
0      2012-13  1610612752               NYK  New York Knicks  0021201218   
1      2012-13  1610612752               NYK  New York Knicks  0021201202   
2      2012-13  1610612752               NYK  New York Knicks  0021201196   
3      2012-13  1610612752               NYK  New York Knicks  0021201182   
4      2012-13  1610612752               NYK  New York Knicks  0021201174   
..         ...         ...               ...              ...         ...   
77     2012-13  1610612752               NYK  New York Knicks  0021200104   
78     2012-13  1610612752               NYK  New York Knicks  0021200071   
79     2012-13  1610612752               NYK  New York Knicks  0021200042   
80     2012-13  1610612752               NYK  New York Knicks  0021200037   
81     2012-13  1610612752               NYK  New York Knicks  0021200020   


10
20
30
40
50
60
70
80
-------------------------------------------------
2015-16
   SEASON_YEAR     TEAM_ID TEAM_ABBREVIATION        TEAM_NAME     GAME_ID  \
0      2015-16  1610612752               NYK  New York Knicks  0021501212   
1      2015-16  1610612752               NYK  New York Knicks  0021501202   
2      2015-16  1610612752               NYK  New York Knicks  0021501180   
3      2015-16  1610612752               NYK  New York Knicks  0021501169   
4      2015-16  1610612752               NYK  New York Knicks  0021501151   
..         ...         ...               ...              ...         ...   
77     2015-16  1610612752               NYK  New York Knicks  0021500063   
78     2015-16  1610612752               NYK  New York Knicks  0021500048   
79     2015-16  1610612752               NYK  New York Knicks  0021500034   
80     2015-16  1610612752               NYK  New York Knicks  0021500019   
81     2015-16  1610612752               NYK  New York Knicks  00215000

10
20
30
40
50
60
70
80
-------------------------------------------------
2018-19
   SEASON_YEAR     TEAM_ID TEAM_ABBREVIATION        TEAM_NAME     GAME_ID  \
0      2018-19  1610612752               NYK  New York Knicks  0021801223   
1      2018-19  1610612752               NYK  New York Knicks  0021801213   
2      2018-19  1610612752               NYK  New York Knicks  0021801204   
3      2018-19  1610612752               NYK  New York Knicks  0021801184   
4      2018-19  1610612752               NYK  New York Knicks  0021801166   
..         ...         ...               ...              ...         ...   
77     2018-19  1610612752               NYK  New York Knicks  0021800054   
78     2018-19  1610612752               NYK  New York Knicks  0021800042   
79     2018-19  1610612752               NYK  New York Knicks  0021800028   
80     2018-19  1610612752               NYK  New York Knicks  0021800018   
81     2018-19  1610612752               NYK  New York Knicks  00218000

10
20
30
40
the lengths
911
911
911
911
    Win/Loss  Matchup  Team Point Average  Opponent Team Point Average
0          L       28          113.000000                   131.000000
1          W       30          113.500000                    98.000000
2          L       16          108.333333                   100.500000
3          L       22          107.000000                   113.666667
4          L       12          106.600000                   106.500000
..       ...      ...                 ...                          ...
906        L       28          105.081081                   113.666667
907        W        6          104.815789                   102.081081
908        W       17          105.461538                   118.921053
909        L       23          105.050000                   115.333333
910        L       12          105.097561                   112.564103

[911 rows x 4 columns]
    Win/Loss  Team Point Average  Opponent Team Point Average  ATL  BKN  BOS  \
0   

In [8]:
backup = test.copy()


In [13]:
test = backup
test

,Win/Loss,Team Point Average,Opponent Team Point Average,ATL,BKN,BOS,CHA,CHI,CLE,DAL,...,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS
0,L,113.000000,131.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,W,113.500000,98.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,L,108.333333,100.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,L,107.000000,113.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,L,106.600000,106.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,L,105.081081,113.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
907,W,104.815789,102.108108,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
908,W,105.461538,119.684211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909,L,105.050000,115.435897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
test["Win/Loss"]

0      L
1      W
2      L
3      L
4      L
      ..
906    L
907    W
908    W
909    L
910    L
Name: Win/Loss, Length: 911, dtype: object

In [10]:
test["Win/Loss"] = test["Win/Loss"].replace("L", 0)
test["Win/Loss"] = test["Win/Loss"].replace("W", 1)
test

,Win/Loss,Team Point Average,Opponent Team Point Average,ATL,BKN,BOS,CHA,CHI,CLE,DAL,...,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS
0,0,113.000000,131.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,113.500000,98.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,108.333333,100.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,107.000000,113.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,106.600000,106.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,0,105.081081,113.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
907,1,104.815789,102.108108,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
908,1,105.461538,119.684211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909,0,105.050000,115.435897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
train_dataset = test.sample(frac=0.90, random_state=0)
test_dataset = test.drop(train_dataset.index)
train_labels = train_dataset.pop("Win/Loss")
test_labels =  test_dataset.pop("Win/Loss")

In [33]:
train_stats = train_dataset.describe()
        
train_stats = train_stats.transpose()
def sigmoid(dataframe):
        
    return((dataframe - train_stats['mean'])/train_stats['std'])
normed_train_data = sigmoid(train_dataset)
normed_test_data = sigmoid(test_dataset)

In [34]:

from tensorflow.keras.layers import Dense

classifier = keras.Sequential()

classifier.add(Dense(units = 32, activation = 'relu', input_dim=31))
classifier.add(Dense(units = 16, activation = 'relu'))
classifier.add(Dense(units = 8, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))


In [45]:
normed_train_data

,Team Point Average,Opponent Team Point Average,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DEN,...,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS
251,-0.365778,-0.341293,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
727,-0.106907,-0.162724,-0.223327,-0.220305,-0.232204,-0.211025,4.880512,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
293,-0.531146,-1.444222,-0.223327,-0.220305,-0.232204,-0.211025,4.880512,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
615,0.486814,0.632062,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,6.670923,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
144,0.925852,-1.164254,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,0.777639,0.257768,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
143,0.946247,-0.689779,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,4.732993,-0.165938,-0.207856
180,-0.443864,-0.190297,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856
131,0.837428,-0.193111,-0.223327,-0.220305,-0.232204,-0.211025,-0.204647,-0.220305,-0.158017,-0.165938,...,-0.15392,-0.217248,-0.235103,-0.149721,-0.169773,-0.162021,-0.162021,-0.211025,-0.165938,-0.207856


In [35]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

In [47]:
classifier.fit(train_dataset, train_labels, batch_size = 1, epochs = 100)

Epoch 1/100
820/820 [==============================] - 1s 691us/step - loss: 39.4534
Epoch 2/100
820/820 [==============================] - 1s 852us/step - loss: 17.8826
Epoch 3/100
820/820 [==============================] - 1s 826us/step - loss: 14.0754
Epoch 4/100
820/820 [==============================] - 1s 699us/step - loss: 15.1306
Epoch 5/100
820/820 [==============================] - 1s 683us/step - loss: 13.7181
Epoch 6/100
820/820 [==============================] - 1s 716us/step - loss: 12.6119
Epoch 7/100
820/820 [==============================] - 1s 705us/step - loss: 12.2629
Epoch 8/100
820/820 [==============================] - 1s 870us/step - loss: 11.0089
Epoch 9/100
820/820 [==============================] - 1s 696us/step - loss: 10.3352
Epoch 10/100
820/820 [==============================] - 1s 714us/step - loss: 9.6561
Epoch 11/100
820/820 [==============================] - 1s 686us/step - loss: 8.7764
Epoch 12/100
820/820 [==============================] - 1s 737us/

In [48]:
y_pred = classifier.predict(test_dataset)

In [16]:
y_pred = [1 if y>= 0.5 else 0 for y in y_pred]


KeyError: 0

In [53]:
print(y_pred)
norm = sigmoid(y_pred[0])
norm

[[4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [6.26565516e-02]
 [5.32901322e-06]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [4.53742206e-01]
 [3.32541240e-05]
 [4.53742206e-01]
 [4.53742206e-01]
 [2.90309489e-02]
 [6.43163919e-04]
 [1.28418446e-01]
 [4.53742206e-01]
 [5.20253181e-03]
 [4.53742206e-01]
 [1.35850696e-05]
 [4.53742206e-01]
 [1.49807662e-12]
 [4.53742206e-01]
 [1.26846363e-11]
 [1.14300847e-03]
 [1.57925270e-10]
 [3.26731801e-03]
 [4.53742206e-01]
 [6.45622489e-09]
 [1.98130583e-05]
 [4.53742206e-01]
 [4.53742206e-01]
 [2.74572153e-06]
 [4.96229529e-03]
 [4.53742206e-01]
 [4.53742206e-01]
 [7.03668993e-05]
 [4.537422

Team Point Average            -20.115350
Opponent Team Point Average   -14.190081
ATL                             1.907264
BKN                             1.936759
BOS                             1.824840
CHA                             2.032284
CHI                             2.102705
CLE                             1.936759
DAL                             2.781649
DEN                             2.640435
DET                             1.936759
GSW                             3.037142
HOU                             2.781649
IND                             1.967374
LAC                             2.860231
LAL                             2.945093
MEM                             2.860231
MIA                             2.102705
MIL                             1.907264
MIN                             2.781649
NOP                             2.781649
OKC                             2.860231
ORL                             1.967374
PHI                             1.799189
PHX             

In [20]:
total = 0
correct = 0
wrong = 0
for i in y_pred:
    total = total +1
    if(test_labels[i] == y_pred[i]):
        correct += 1
    else:
        wrong +=1
print(total)
print(correct)
print(wrong)

KeyError: 0